In [ ]:
## Exercise 1.)
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV



s3= boto3.resource('s3')
bucket_name= 'macklins-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'CarPrice_Assignment.csv'

bucket_object= bucket.Object(file_key)
file_object= bucket_object.get()
file_content_stream= file_object.get('Body')

## Reading the data file
car_price= pd.read_csv(file_content_stream)
car_price.head()

In [6]:
# defining the input and target variables
X = car_price[['wheelbase', 'enginesize', 'horsepower', 'compressionratio', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

# splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [7]:
# estimating lambda for lasso
lasso_cv = LassoCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], normalize =True, cv= 5).fit(X_train, Y_train)

# extracting the best lamba
cv_lambda = lasso_cv.alpha_
print('The estimated lamba for the lasso model is', cv_lambda)

# build lasso
lasso_md = Lasso(alpha = cv_lamba, normalize = True).fit(X_train, Y_train)
lasso_md.coef_

The estimated lamba for the lasso model is 1.0


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

array([ 161.99830787,  104.45437452,   58.51321121,  308.98120796,
          1.86105372, -170.14107375,   73.40239866])

In [8]:
# dropping highwaympg
X_train= X_train.drop(columns= ['highwaympg'], axis= 1)
X_test= X_test.drop(columns= ['highwaympg'], axis= 1)


def l2_normalization(X):
    
    x_mean= np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2


X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [9]:
# linear regression
lm_md = LinearRegression().fit(X_train, Y_train)

# predicting on test data
lm_pred = lm_md.predict(X_test)

# computing the mse of the linear regression model
mse1 = np.mean(np.power(Y_test - lm_pred, 2))
print('The mse of the linear model is:', mse1)

The mse of the linear model is: 15683929.44186365


In [14]:
# ridge regression
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv =5).fit(X_train, Y_train)

# extracting the best lamba
cv_lambda= ridge_cv.alpha_
print('The best lamba of the ridge model is:', cv_lambda)

# building the ridge model
ridge_md= Ridge(alpha = cv_lambda).fit(X_train, Y_train)

# predicting on test
ridge_pred = ridge_md.predict(X_test)

# computing the mse of the ridge regression model
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is:', mse2)

The best lamba of the ridge model is: 0.001
The mse of the ridge model is: 17700940.257173933


In [ ]:
# The mse of model 1 is smaller, meaning that model 1 is preferred.